# 下載 Pretrained model
https://github.com/deepinsight/insightface/wiki/Model-Zoo#34-mobilefacenetarcfacems1m-refine-v1

In [5]:
!wget https://www.dropbox.com/s/akxeqp99jvsd6z7/model-MobileFaceNet-arcface-ms1m-refine-v1.zip
!unzip model-MobileFaceNet-arcface-ms1m-refine-v1.zip 
!rm model-MobileFaceNet-arcface-ms1m-refine-v1.zip

--2020-11-05 16:52:28--  https://www.dropbox.com/s/akxeqp99jvsd6z7/model-MobileFaceNet-arcface-ms1m-refine-v1.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.1, 2620:100:6030:1::a27d:5001
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/akxeqp99jvsd6z7/model-MobileFaceNet-arcface-ms1m-refine-v1.zip [following]
--2020-11-05 16:52:28--  https://www.dropbox.com/s/raw/akxeqp99jvsd6z7/model-MobileFaceNet-arcface-ms1m-refine-v1.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbaa0c510cdd19ab8ed8fe2332a.dl.dropboxusercontent.com/cd/0/inline/BCkk5e9bCWBA5XQ3PLEZYl5Hx9999MW4lZGghYsygS6KNj24sbdFqHR7jg0maS5in7ovpy5zNnzgpR2IcySAaJk8FSYJ1WDwFkcoXlMQFWmweQ/file# [following]
--2020-11-05 16:52:29--  https://ucbaa0c510cdd19ab8ed8fe2332a.dl.dropboxusercontent.com/cd/0/inline/BCkk5e9bCWBA5XQ3PLEZYl5Hx9999M

In [3]:
!pip install -q onnx==1.2.1
!pip install -q --user mxnet==1.5.1

### 安裝完須重啟 kernel
由於我們使用 --user mxnet 安裝，因此會裝在 ~/.local/lib/python3.7/site-packages 裡，方便我們修改這兩個檔案
- mxnet/contrib/onnx/mx2onnx/_op_translations.py
- mxnet/contrib/onnx/onnx2mx/_op_translations.py

In [1]:
!cp -rf mx2onnx/_op_translations.py ~/.local/lib/python3.7/site-packages/mxnet/contrib/onnx/mx2onnx
!cp -rf onnx2mx/_op_translations.py ~/.local/lib/python3.7/site-packages/mxnet/contrib/onnx/onnx2mx

In [2]:
import cv2
import mxnet as mx
import numpy as np
from mxnet.contrib import onnx as onnx_mxnet
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
sym = 'model-y1-test2/model-symbol.json'
params = 'model-y1-test2/model-0000.params'
input_shape = (1, 3, 112, 112)
onnx_file = './mobilefacenet.onnx'

In [6]:
converted_model_path = onnx_mxnet.export_model(sym, params, [input_shape], np.float32, onnx_file)

INFO:root:Converting json and weight file to sym and params
INFO:root:Input shape of the model [(1, 3, 112, 112)] 
INFO:root:Exported ONNX file ./mobilefacenet.onnx saved to disk


In [7]:
import onnx
import onnxruntime as rt
from onnx import checker

# Load onnx model
model = onnx.load_model(converted_model_path)

# Check if converted ONNX protobuf is valid
checker.check_graph(model.graph)

In [8]:
sess = rt.InferenceSession(converted_model_path)

In [9]:
input_name = sess.get_inputs()[0].name
print("input name", input_name)
input_shape = sess.get_inputs()[0].shape
print("input shape", input_shape)
input_type = sess.get_inputs()[0].type
print("input type", input_type)

input name data
input shape [1, 3, 112, 112]
input type tensor(float)


In [10]:
output_name = sess.get_outputs()[0].name
print("output name", output_name)
output_shape = sess.get_outputs()[0].shape
print("output shape", output_shape)
output_type = sess.get_outputs()[0].type
print("output type", output_type)

output name fc1
output shape [1, 128]
output type tensor(float)


In [11]:
x = np.random.random([1, 3, 112, 112]).astype(np.float32)
res = sess.run([output_name], {input_name: x})
print(res)

[array([[ 0.01577088, -0.41153952, -0.07544725,  0.7216799 , -0.1290518 ,
         0.07569689, -0.16506198, -0.1542044 ,  0.04309328, -0.21662244,
         0.04711505,  0.11998217, -0.23235516, -0.36025846, -0.17884608,
        -0.13206078,  0.16468489,  0.25729367, -0.32697445, -0.10646572,
        -0.02304216, -0.4696744 , -0.09949888,  0.13706475, -0.13745508,
         0.26182014, -0.2015687 ,  0.13166358, -0.23082791,  0.19544603,
         0.10573763, -0.0872464 , -0.04051244,  0.24559316,  0.18936524,
         0.15481204, -0.27061376,  0.53524226,  0.5201192 , -0.34306094,
        -0.5053572 ,  0.45167515,  0.3159355 ,  0.01370022, -0.2678917 ,
        -0.14117633, -0.03223775,  0.37744275, -0.4613757 , -0.13261105,
        -0.02520673, -0.73135436, -0.5750462 ,  0.0397728 , -0.006244  ,
        -0.23068933,  0.06726556,  0.04545179, -0.07261792, -0.01804775,
        -0.0401057 ,  0.05102379,  0.43539938, -0.25982893, -0.23811403,
         0.42094254, -0.05543632, -0.21422154, -0.